In [1]:
#import packages
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import sql

In [7]:
data_path = './validated/movie_data_validated_0.csv'
data = pd.read_csv(data_path)

In [8]:
data

,movie_title,title_year,language,country,duration,content_rating,genres,movie_imdb_link,director_name,imdb_score,num_voted_users,gross,actors
0,Avatar,2009,English,USA,178.0,PG-13,Action|Adventure|Fantasy|Sci-Fi,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,James Cameron,7.9,886204,760505847.0,CCH Pounder|Joel David Moore|Wes Studi
1,Pirates of the Caribbean: At World's End,2007,English,USA,169.0,PG-13,Action|Adventure|Fantasy,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,Gore Verbinski,7.1,471220,309404152.0,Johnny Depp|Orlando Bloom|Jack Davenport
2,Spectre,2015,English,UK,148.0,PG-13,Action|Adventure|Thriller,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,Sam Mendes,6.8,275868,200074175.0,Christoph Waltz|Rory Kinnear|Stephanie Sigman
3,The Dark Knight Rises,2012,English,USA,164.0,PG-13,Action|Thriller,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,Christopher Nolan,8.5,1144337,448130642.0,Tom Hardy|Christian Bale|Joseph Gordon-Levitt
4,John Carter,2012,English,USA,132.0,PG-13,Action|Adventure|Sci-Fi,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,Andrew Stanton,6.6,212204,73058679.0,Daryl Sabara|Samantha Morton|Polly Walker
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539,El Mariachi,1992,Spanish,USA,81.0,R,Action|Crime|Drama|Romance|Thriller,http://www.imdb.com/title/tt0104815/?ref_=fn_t...,Robert Rodriguez,6.9,52055,2040920.0,Carlos Gallardo|Peter Marquardt|Consuelo Gómez
4540,The Mongol King,2005,English,USA,84.0,PG-13,Crime|Drama,http://www.imdb.com/title/tt0430371/?ref_=fn_t...,Anthony Vallone,7.8,36,NaN,Richard Jewell|John Considine|Sara Stepnicka
4541,Newlyweds,2011,English,USA,95.0,Not Rated,Comedy|Drama,http://www.imdb.com/title/tt1880418/?ref_=fn_t...,Edward Burns,6.4,1338,4584.0,Kerry Bishé|Caitlin FitzGerald|Daniella Pineda
4542,Shanghai Calling,2012,English,USA,100.0,PG-13,Comedy|Drama|Romance,http://www.imdb.com/title/tt2070597/?ref_=fn_t...,Daniel Hsia,6.3,1255,10443.0,Alan Ruck|Daniel Henney|Eliza Coupe


In [9]:
initial_row_count = data.shape[0]

data = data.dropna(subset='actors')

final_row_count = data.shape[0]
rows_deleted = initial_row_count - final_row_count
print(f"Total rows deleted: {rows_deleted}")

Total rows deleted: 1


## connect to gres

In [12]:
params = {
    'database': 'kubedb',
    'user': 'db_user',
    'password': 'user@kube',
    'host': '127.0.0.1',
    'port': 5432
}

## Create table

In [13]:
conn = psycopg2.connect(**params)
cur = conn.cursor()

### movies

In [20]:
create_table_query = '''
CREATE TABLE IF NOT EXISTS movie (
    id SERIAL PRIMARY KEY,
    movie_title TEXT, /*string*/
    title_year INT,
    language TEXT,
    country TEXT,
    duration INT,
    content_rating TEXT,
    movie_imdb_link TEXT,
    director_name TEXT,
    imdb_score REAL, /*float*/
    num_voted_users INT,
    gross REAL
);
'''
cur.execute(create_table_query)
conn.commit()

In [ ]:
#TODO: print columns
# table_schema_query = '''
#     SELECT table_schema
#     FROM information_schema.tables
#     WHERE table_name = %s
# '''
# table = 'movies'
# cur.execute(table_schema_query, (table,))
# 
# schema = cur.fetchall()
# if schema:
#     print(schema[0])

In [ ]:
# cur.execute("SELECT * FROM movies")
# rows = cur.fetchall()
# for row in rows:
#     print(row)

In [ ]:
# cur.execute('''DROP TABLE genres CASCADE ''')
# conn.commit()

### Genre

In [21]:
create_table_query = '''
CREATE TABLE IF NOT EXISTS genre (
    id SERIAL PRIMARY KEY,
    name TEXT UNIQUE /* UNIQUE: 每一行都必须有一个唯一的值，不允许有重复*/
);
'''
cur.execute(create_table_query)
conn.commit()

In [22]:
create_table_query = '''
CREATE TABLE IF NOT EXISTS movie_genre (
    movie_id INT REFERENCES movie(id),
    genre_id INT REFERENCES genre(id),
    PRIMARY KEY (movie_id, genre_id)
);
'''
cur.execute(create_table_query)
conn.commit()

### Actor

In [23]:
create_table_query = '''
CREATE TABLE IF NOT EXISTS actor (
    id SERIAL PRIMARY KEY,
    name TEXT UNIQUE
);
'''
cur.execute(create_table_query)
conn.commit()

In [24]:
create_table_query = '''
CREATE TABLE IF NOT EXISTS movie_actor (
    movie_id INT REFERENCES movie(id),
    actor_id INT REFERENCES actor(id),
    PRIMARY KEY (movie_id, actor_id)
);
'''
cur.execute(create_table_query)
conn.commit()

In [25]:
cur.close()
conn.close()

## Insert data

In [26]:
conn = psycopg2.connect(**params)
cur = conn.cursor()

### movie

In [27]:
my_table = 'movie'
insert_columns = ['movie_title', 'title_year', 'language', 'country', 'duration', 'content_rating', 'movie_imdb_link', 'director_name', 'imdb_score', 'num_voted_users', 'gross']

# Building the query string safely
column_identifiers = [sql.Identifier(column) for column in insert_columns]
insert_query = sql.SQL(
    'INSERT INTO {table} ({fields}) '
    'VALUES ({values})'
).format(
    table=sql.Identifier(my_table),
    fields=sql.SQL(', ').join(map(sql.Identifier, insert_columns)),
    values=sql.SQL(', ').join(sql.Placeholder() * len(insert_columns))
)

#data
data_to_insert = data[insert_columns]

# Executing the query with a cursor
for i, row in data_to_insert.iterrows():
    values = [row[col] for col in insert_columns]
    cur.execute(insert_query, values)
    
conn.commit()

### genres

In [28]:
unique_genres = data['genres'].str.get_dummies(sep='|').columns.tolist()
print(unique_genres)

['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']


In [29]:
for genre in unique_genres:
    cur.execute("""
        INSERT INTO genre (name)
        VALUES (%s)
        ON CONFLICT (name) DO NOTHING;
    """, (genre,))
    
conn.commit()

In [30]:
cur.execute("SELECT * FROM genre")
genre_id = cur.fetchall()
print(genre_id)

[(1, 'Action'), (2, 'Adventure'), (3, 'Animation'), (4, 'Biography'), (5, 'Comedy'), (6, 'Crime'), (7, 'Documentary'), (8, 'Drama'), (9, 'Family'), (10, 'Fantasy'), (11, 'Film-Noir'), (12, 'History'), (13, 'Horror'), (14, 'Music'), (15, 'Musical'), (16, 'Mystery'), (17, 'News'), (18, 'Romance'), (19, 'Sci-Fi'), (20, 'Sport'), (21, 'Thriller'), (22, 'War'), (23, 'Western')]


#### movie-genre

In [31]:
for i, row in data.iterrows():
    movie_title = row['movie_title']
    genres = row['genres'].split('|')
    
    for genre in genres:
        # 获取当前movie的id
        cur.execute("""SELECT id FROM movie WHERE movie_title = %s;""", (movie_title,))
        movie_id = cur.fetchone()[0]
        
        # 获取当前genre的id
        cur.execute("""SELECT id FROM genre WHERE name = %s;""", (genre,))
        genre_id = cur.fetchone()[0]
        
        # 插入movie和genre的关联到movie_genre_association表
        cur.execute("""
            INSERT INTO movie_genre (movie_id, genre_id)
            VALUES (%s, %s)
            ON CONFLICT DO NOTHING;
        """, (movie_id, genre_id))

# 提交事务至数据库
conn.commit()

### Actor

In [32]:
unique_actors = data['actors'].str.get_dummies(sep='|').columns.tolist()
print(unique_actors)

['50 Cent', 'A. Michael Baldwin', 'A.J. Buckley', 'A.J. Langer', 'AJ Michalka', 'Aaliyah', 'Aaron Ashmore', 'Aaron Hughes', 'Aaron Stanford', 'Aaron Staton', 'Aaron Yoo', 'Aasheekaa Bathija', 'Aasif Mandvi', 'Abbie Cornish', 'Abby Elliott', 'Abby Mukiibi Nkaaga', 'Abhishek Bachchan', 'Abigail Spencer', 'Abraham Benrubi', 'Adam Alexi-Malle', 'Adam Arkin', 'Adam Baldwin', 'Adam Boyer', 'Adam Brown', 'Adam Butcher', 'Adam Carolla', 'Adam Copeland', 'Adam Garcia', 'Adam Goldberg', 'Adam Hicks', 'Adam Lamberg', 'Adam LeFevre', 'Adam Ratcliffe', 'Adam Rayner', 'Adam Sandler', 'Adam Scott', 'Adam Trese', 'Adel Karam', 'Adelaide Kane', 'Aden Young', 'Adhir Kalyan', 'Aditya Roy Kapoor', 'Adolphe Menjou', 'Adrian Alonso', 'Adrian Gonzalez', 'Adrian Holmes', 'Adrian Lester', 'Adrian Martinez', 'Adrian Pasdar', 'Adrian Paul', 'Adriana Barraza', 'Adriana Caselotti', 'Adriane Lenox', 'Adriano Giannini', 'Adrien Dorval', 'Adrienne Barbeau', 'Adrienne Frantz', 'Adrienne King', 'Adrienne Pickering', 'A

In [33]:
for actor in unique_actors:
    cur.execute("""
        INSERT INTO actor (name)
        VALUES (%s)
        ON CONFLICT (name) DO NOTHING;
    """, (actor,))
    
conn.commit()

In [34]:
cur.execute("""SELECT * FROM actor""")
results = cur.fetchall()
df_actor = pd.DataFrame(results)

In [35]:
df_actor

,0,1
0,1,50 Cent
1,2,A. Michael Baldwin
2,3,A.J. Buckley
3,4,A.J. Langer
4,5,AJ Michalka
...,...,...
5597,5598,Zoë Poledouris
5598,5599,Zubaida Sahar
5599,5600,Álex Angulo
5600,5601,Émilie Dequenne


#### movie-actor

In [36]:
for i, row in data.iterrows():
    movie_title = row['movie_title']
    actors = row['actors'].split('|')
    
    for actor in actors:
        # 获取当前movie的id
        cur.execute("""SELECT id FROM movie WHERE movie_title = %s;""", (movie_title,))
        movie_id = cur.fetchone()[0]
        
        # 获取当前actor的id
        cur.execute("""SELECT id FROM actor WHERE name = %s;""", (actor,))
        actor_id = cur.fetchone()[0]
        
        # 插入 movie 和 actor 的关联到movie_actor表
        cur.execute("""
            INSERT INTO movie_actor (movie_id, actor_id)
            VALUES (%s, %s)
            ON CONFLICT DO NOTHING;
        """, (movie_id, actor_id))

# 提交事务至数据库
conn.commit()

In [37]:
cur.execute("""SELECT * FROM movie_actor""")
results = cur.fetchall()
df_movie_actor = pd.DataFrame(results)

In [38]:
df_movie_actor

,0,1
0,1,725
1,1,2572
2,1,5469
3,2,2662
4,2,4070
...,...,...
13616,4542,1153
13617,4542,1534
13618,4543,2593
13619,4543,648


In [39]:
# 关闭游标和连接
cur.close()
conn.close()